In [1]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import math
import pandas as pd

import random
import logging
logging.basicConfig(level=logging.INFO)


def init_spark():
    spark = SparkSession \
            .builder \
            .config("spark.driver.memory", "50g") \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .getOrCreate()
    return spark

spark = init_spark()

df2 = pd.read_csv("C:/Users/vansh/Desktop/Poject big data/anime.csv")
df = spark.read.text("C:/Users/vansh/Desktop/Poject big data/sample1.csv").rdd
data = df.map(lambda x: x[0].split(','))
header1=data.first()
data=data.filter(lambda x:x !=header1)
data2 = data.groupBy(lambda x : x[1])
data3 = data2.collect()

#-------------------------------------------------------------------------------------------------------------------------------




#all_data = data.map(lambda x:[int(x[1]),[int(x[0]),int(x[2])]]).collect()



utility = {}
Anime_id = []
user_id = []

for user in range(len(data3)):

    utility[int(data3[user][0])] = {}
    #if int(data3[user][0]) not in all_users:
    #    all_users.append(int(i[0]))
    for i in data3[user][1]:

        if int(i[0]) not in user_id:
          user_id.append(int(i[0]))

        if int(i[2]) == -1:
            continue
        utility[int(i[1])][int(i[0])] = int(i[2])


Anime_id= list(utility.keys())




In [2]:
utility

{20: {3: 8,
  5: 6,
  21: 8,
  28: 9,
  34: 9,
  38: 6,
  39: 10,
  41: 10,
  43: 8,
  46: 7,
  50: 10,
  53: 6,
  60: 9,
  61: 7,
  62: 7,
  72: 8,
  79: 9,
  80: 9,
  88: 9,
  95: 7,
  96: 10,
  97: 6,
  99: 5,
  101: 10,
  104: 9,
  108: 6,
  112: 10,
  116: 10,
  118: 7,
  124: 7,
  125: 4,
  128: 8,
  129: 9,
  130: 10,
  132: 9,
  140: 7,
  148: 8,
  153: 6,
  159: 7,
  163: 9,
  164: 9,
  166: 6,
  171: 10,
  173: 10,
  177: 9,
  179: 6,
  185: 7,
  187: 6,
  189: 8,
  190: 9,
  191: 7,
  195: 7,
  196: 7,
  199: 9,
  202: 6,
  209: 9,
  210: 10,
  214: 7,
  219: 9,
  220: 10,
  226: 7,
  234: 5,
  235: 8,
  236: 7,
  241: 8,
  246: 9,
  248: 9,
  249: 8,
  250: 7,
  252: 8,
  253: 6,
  258: 9,
  260: 9,
  261: 7,
  263: 7,
  268: 8,
  269: 7,
  271: 1,
  272: 8,
  273: 5,
  275: 9,
  279: 7,
  281: 8,
  282: 9,
  284: 7,
  288: 8,
  290: 9,
  293: 8,
  294: 10,
  296: 10,
  297: 10,
  298: 9,
  311: 10,
  312: 9,
  316: 9,
  317: 9,
  322: 9,
  325: 8,
  329: 7,
  332: 5,
  334

In [3]:
def sim2(matrix, item1, item2):
    matched = []

    for user in list(matrix[item1].keys()):





        if user in list(matrix[item2].keys()):
            if user not in matched:

                matched.append(user)






    x = []
    y = []

    for user in matched:

            x.append(matrix[item1][int(user)])
            y.append(matrix[item2][int(user)])

    f = (lambda i, j: i - j)

    x = f(x, np.mean(x))

    y = f(y, np.mean(y))

    #Pearson Correlation Coefficient
    numerator = (lambda x, y : x*y)

    denominator = (lambda x: x*x)
    d = math.sqrt(sum(denominator(x))) * math.sqrt(sum(denominator(y)))

    if d == 0:
        return 0
    similarity = sum(numerator(x, y))/ d
    return similarity




def most_similar_item(matrix, item, n=10):
    similarity = {}
    for p in list(matrix.keys()):


        similarity[p] = sim2(matrix, item, p)
    sorted_list = sorted(similarity.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]



def recommend(matrix, item):
    similar_items = most_similar_item(matrix, item)


    predictions = {}
    for user in user_id:

        num = 0
        den = 0
        user=int(user)
        for item in similar_items:

            if  user in matrix[item[0]].keys():


                num +=   int(matrix[item[0]][user]) * int(item[1])
                den += int(item[1])


        if den == 0:
            continue
        if user in matrix[item[0]].keys():
            predictions[user] = float(num/den)
    return predictions

test= int(0.05 * len(Anime_id))


test_data_item = []
for _ in range(50):
    random_item = random.randint(1,  2000)
    if Anime_id[random_item] not in test_data_item:
        test_data_item.append(Anime_id[random_item])

test_user = int(0.1 * len(user_id))
test_data_user = []
for _ in range(test_user):
    random_user = random.randint(1, len(user_id)-1)
    if user_id[random_user] not in test_data_user:
        test_data_user.append(user_id[random_user])


import copy
training_data = copy.deepcopy(utility)

for anime in test_data_item:
    for user in test_data_user:
        user=int(user)
        if (user) in training_data[anime].keys():

            del training_data[anime][user]


def start():
    predicted = {}

    for item in test_data_item:

        predicted[item]=recommend(training_data, item)


    return predicted



predicted = start()



def RMSE(actual, predicted):
    error = 0
    n = 0
    for anime in test_data_item:

        for user in test_data_user:

            if user in list(predicted[anime].keys()) and user in list(actual[anime].keys()):

                error += (int(actual[anime][user]) - int(predicted[anime][user]))**2
                n += 1

    return math.sqrt(error/n)
print(RMSE(utility, predicted))

C:\Users\vansh\Miniconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\vansh\Miniconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.0


In [4]:
predictions =[]
def recommend_users(for_animde_id,n=10):
    return recommend(training_data,for_animde_id)
for anime_id in Anime_id:

      predictions.append([anime_id,recommend_users(anime_id)])

print(predictions)

final=[]
final_predictions={}

[[20, {392: 5.0, 418: 7.666666666666667, 462: 6.0}], [24, {5: 2.2, 226: 8.0, 271: 7.333333333333333, 282: 8.8, 379: 7.25, 404: 7.0, 405: 7.666666666666667, 443: 10.0, 492: 4.571428571428571, 497: 9.0, 120: 9.0, 256: 8.333333333333334, 321: 7.833333333333333, 446: 6.25, 17: 6.5, 326: 9.0, 7: 7.166666666666667}], [79, {53: 8.5, 79: 10.0, 80: 10.0, 177: 8.0, 191: 8.0, 236: 7.0, 250: 10.0, 271: 5.75, 282: 9.0, 293: 8.0, 294: 8.0, 354: 8.0, 392: 5.25, 395: 8.0, 478: 8.428571428571429, 183: 6.5, 186: 9.0, 438: 8.0, 256: 9.166666666666666, 461: 8.0, 37: 9.0, 155: 8.0, 244: 8.666666666666666, 326: 10.0, 348: 5.0, 500: 9.0, 301: 7.333333333333333, 373: 7.5, 233: 10.0, 87: 9.0, 350: 10.0, 428: 9.0}], [226, {271: 7.666666666666667, 17: 6.6, 462: 7.0, 123: 7.333333333333333}], [241, {5: 4.0, 41: 9.0, 317: 6.5, 339: 8.0, 492: 6.333333333333333, 320: 9.0, 321: 9.0, 446: 9.0, 17: 5.0, 341: 7.5, 371: 8.0, 7: 7.25, 433: 9.0}], [355, {43: 7.333333333333333, 177: 7.0, 271: 8.5, 322: 7.0, 325: 6.0, 392: 5

In [5]:
for user in user_id:
    for it in predictions:
        if int(user) in it[1].keys():
            final.append([it[0],it[1].get((user))])


    final_predictions[user]=final
    final=[]



print(final_predictions)


{1: [[12361, 10.0], [15847, 10.0], [6920, 10.0], [10260, 10.0], [1655, 10.0], [461, 10.0], [3323, 10.0], [109, 10.0], [10380, 10.0], [3247, 10.0], [2391, 10.0], [2580, 10.0]], 3: [[2993, 7.0], [4581, 10.0], [10076, 6.5], [11266, 8.0], [20045, 6.0], [154, 7.5], [341, 6.0], [552, 6.0], [1121, 8.0], [1122, 8.0], [1764, 6.0], [9107, 6.8], [14093, 8.666666666666666], [16918, 6.0], [18097, 10.0], [18393, 7.0], [21507, 8.0], [17505, 9.0], [137, 8.0], [138, 9.0], [248, 8.0], [508, 9.0], [534, 10.0], [550, 9.0], [898, 8.0], [1669, 7.5], [2104, 7.666666666666667], [2248, 7.0], [2397, 7.5], [3887, 8.666666666666666], [3974, 7.5], [4106, 7.0], [4472, 8.666666666666666], [6974, 9.0], [8457, 8.0], [9741, 7.4], [11371, 8.0], [13201, 9.0], [13535, 8.0], [15335, 10.0], [19953, 7.75], [21185, 7.666666666666667], [24781, 8.333333333333334], [1311, 7.75], [2994, 7.5], [2026, 10.0], [9063, 7.0], [9958, 7.333333333333333], [21325, 6.5], [25687, 10.0], [30347, 8.0], [2581, 7.0], [338, 8.5], [1709, 8.0], [268